In [18]:
import numpy as np
import pandas as pd
import geopandas as gpd
import operator

import matplotlib
import matplotlib.pyplot as plt

import os
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource
from bokeh.models.tools import HoverTool
from bokeh import palettes

import bokeh.models as bmo

In [4]:
output_notebook()

Loading BokehJS ...

In [5]:
data_path = 'Romain/data/'
#data_path = '../Data/'

In [6]:
path_to_datasets = data_path + 'related_datasets/'

In [7]:
inputs_path = data_path + 'inputs/'
intermediate_path = data_path + 'intermediate/'
output_path = data_path + 'outputs/with_irrig/'

base_path = inputs_path + 'Base/'
diet_path = inputs_path + 'diet/'
pop_path = inputs_path + 'population/'

data_2000_no_irrig_path = intermediate_path + 'Baseline_df_iterations/'
data_2000_path = intermediate_path + 'Baseline_df_iterations/with_irrig/'

#data_2050_no_irrig_path = intermediate_path + 'Future_dfs/All_change/'
data_2050_path = intermediate_path + 'Future_dfs/All_change+irrig/'

data_2050_gdp_cont_path = data_2050_path + 'gdp_cont/original/'
data_2050_gdp_income_path = data_2050_path + 'gdp_cat_income/original/'
data_2050_gdp_stat_path = data_2050_path + 'gdp_cat_stat/original/'

data_2050_gdp_cont_clustered_path = data_2050_path + 'gdp_cont/clustered/'
data_2050_gdp_income_clustered_path = data_2050_path + 'gdp_cat_income/clustered/'
data_2050_gdp_stat_clustered_path = data_2050_path + 'gdp_cat_stat/clustered/'

model_output_path = output_path + 'model_output/'
composite_path = output_path + 'composite/'
compare_path = output_path + 'compare/'
suff_path = output_path + 'sufficiency/'


figure_path = 'figures/with_irrig/'
dist_figures_path = figure_path + 'distributions/'
agg_assumption_fig_path = figure_path + 'agg_assumption/'
cluster_fig_path = figure_path + 'clusters/'
convex_hull_fig_path = figure_path + 'convex_hull/'
suff_fig_path = figure_path + 'sufficiency/'
model_fig_path = figure_path + 'models/'
sensitivity_fig_path = figure_path + 'sensitivity/'

model_path = data_path + 'models/'

# Import & Join datasets

## Sufficiencies

In [8]:
path_to_suff = os.path.join(path_to_datasets,'country_sufficiencies.csv')
suff = pd.read_csv(path_to_suff)

In [9]:
suff.head()

,ISO3,2000,ssp1,ssp2,ssp3,ssp4,ssp5
0,AFG,0.712556,0.536646,0.468967,0.390449,0.367319,0.655464
1,AGO,0.591377,0.453087,0.571087,0.381656,0.558107,0.686487
2,ALB,2.329763,2.544620,2.428040,2.353029,2.508106,2.448250
3,ARE,0.417912,0.513526,0.515186,0.525990,0.529507,0.520904
4,ARG,7.931488,5.890471,6.701443,6.249030,6.205168,6.367101


## Import independency

In [8]:
path_to_gpck = os.path.join(path_to_datasets,'Import_independency/Matti_Kummu_2019/shp_trade_dep.gpkg')
import_indep = gpd.read_file(path_to_gpck)#, layer='countries')

In [9]:
imports_data = import_indep[['cntry', 'admin','country','yr2013']]
imports_data.head()

,cntry,admin,country,yr2013
0,None,Saint Barthelemy,None,NaN
1,None,Jersey,None,NaN
2,None,Ashmore and Cartier Islands,None,NaN
3,None,Siachen Glacier,None,NaN
4,None,Indian Ocean Territories,None,NaN


In [10]:
countries_names = pd.read_csv(base_path + 'country_names.csv',encoding='latin-1')
countries_names.head()

,id,ISO3,adm0_a3,name,nev_admin_name,fao_producer_prices_name,nev_name,nev_name_long,sovereignt,base_id,...,iana_internet,un_vehicle,ioc_olympic,un_iso_numeric,itu_calling,UNI,UNDP,FAOSTAT,GAU,FAOSTAT_augmented
0,1,ABW,ABW,Aruba,Aruba,Aruba,Aruba,Aruba,Netherlands,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1533
1,2,AFG,AFG,Afghanistan,Afghanistan,Afghanistan,Afghanistan,Afghanistan,Afghanistan,1,...,.af,AFG,AFG,4.0,93,4.0,AFG,2.0,1.0,2
2,3,AGO,AGO,Angola,Angola,Angola,Angola,Angola,Angola,2,...,.ao,NaN,ANG,24.0,244,24.0,ANG,7.0,8.0,7
3,4,AIA,AIA,Anguilla,Anguilla,Anguilla,Anguilla,Anguilla,United Kingdom,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1660
4,5,ALB,ALB,Albania,Albania,Albania,Albania,Albania,Albania,4,...,.al,AL,ALB,8.0,355,8.0,ALB,3.0,3.0,3


In [11]:
# merge import with country_names to add ISO3 columns
imports = imports_data.merge(countries_names[['ISO3', 'name']], left_on='admin', right_on='name', how='right')
imports.head()

,cntry,admin,country,yr2013,ISO3,name
0,None,Saint Barthelemy,None,NaN,BLM,Saint Barthelemy
1,None,Jersey,None,NaN,JEY,Jersey
2,None,Siachen Glacier,None,NaN,KAS,Siachen Glacier
3,None,Indian Ocean Territories,None,NaN,IOA,Indian Ocean Territories
4,None,South Georgia and South Sandwich Islands,None,NaN,SGS,South Georgia and South Sandwich Islands


In [12]:
# Join with Sufficiencies
sufficiencies = suff.merge(imports, on='ISO3').drop(['admin', 'country', 'cntry'], axis=1)
sufficiencies = sufficiencies.rename({'yr2013': 'trade_dep_2013'}, axis=1)
sufficiencies.head()

,ISO3,2000,ssp1,ssp2,ssp3,ssp4,ssp5,trade_dep_2013,name
0,AFG,0.712556,0.536646,0.468967,0.390449,0.367319,0.655464,0.269311,Afghanistan
1,AGO,0.591377,0.453087,0.571087,0.381656,0.558107,0.686487,0.221301,Angola
2,ALB,2.329763,2.544620,2.428040,2.353029,2.508106,2.448250,0.208123,Albania
3,ARE,0.417912,0.513526,0.515186,0.525990,0.529507,0.520904,0.000000,United Arab Emirates
4,ARG,7.931488,5.890471,6.701443,6.249030,6.205168,6.367101,1.000000,Argentina


In [13]:
# Check number of countries where we have no info about trade
len(suff), len(sufficiencies.dropna(how='any'))

(166, 148)

In [14]:
# Check which country it is
nan = [c for c in list(suff['ISO3']) if c not in list(sufficiencies.dropna(how='any')['ISO3'])]
nan

['BDI',
 'BTN',
 'CIV',
 'COD',
 'ERI',
 'GNQ',
 'LBY',
 'OMN',
 'PNG',
 'PRI',
 'PSX',
 'SDS',
 'SGP',
 'SOL',
 'SOM',
 'SRB',
 'SYR',
 'TZA']

## Water Security 

* **Source**: Brauman, K. A., B. D. Richter, S. Postel, M. Malsy and M. Flörke (2016). "Water depletion: An improved metric for incorporating seasonal and dry-year water scarcity into water risk assessments." Elementa: Science of the Anthropocene 4(1): 000083

* **Aggregation by country**: aggregate it by country as the %area in any of the depleted category (i.e Categories: 5 to 8 )—> Looking at column OrigDepletionCategory


In [15]:
path_to_tif = os.path.join(path_to_datasets,'Water_Security/GlobalWaterScarcity/DepletionMapCurrent.tif')

In [16]:
# tif to tabular
#! gdal_translate Romain/data/related_datasets/Water_Security/GlobalWaterScarcity/DepletionMapCurrent.tif Romain/data/related_datasets/Water_Security/GlobalWaterScarcity/DepletionMapCurrent.xyz

In [17]:
file = pd.read_csv(path_to_datasets + 'Water_Security/GlobalWaterScarcity/' + 'DepletionMapCurrent.xyz', delimiter=' ', header=None)
new_df = pd.DataFrame(file[2].values, columns=['water_sec'])
new_df.to_csv(path_to_datasets + 'Water_Security/GlobalWaterScarcity/' + 'water_security.csv')

In [18]:
# Aggregate
countries = pd.read_csv(base_path + 'country_ids.csv')
countries_names = pd.read_csv(base_path + 'country_names.csv', encoding='latin-1')
countries = countries[['country_id','pixel_id']].merge(countries_names[['base_id','ISO3','name', 'sovereignt']],right_on='base_id',left_on='country_id',how='right')
countries = countries.set_index('pixel_id')

water_sec = pd.read_csv(path_to_datasets + 'Water_Security/GlobalWaterScarcity/' + 'water_security.csv', index_col=0)
water_sec.index.name = 'pixel_id'
water_sec_countries = water_sec.join(countries, on='pixel_id').dropna()

/Users/Romain/anaconda/envs/project/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [19]:
# count number of values in each category per country
diff_water_sec_countries = water_sec_countries.groupby(['ISO3', 'water_sec'])['water_sec'].count().rename('water_sec_count').reset_index(level=1)
# keep only categories >= 5 and sum value counts per country
interesting = diff_water_sec_countries[diff_water_sec_countries['water_sec'] >= 5].groupby('ISO3')['water_sec_count'].sum()

# total number of values per country
total_water_sec_countries = water_sec_countries.groupby(['ISO3'])['ISO3'].count().rename('total_count')

# compute water sec val
joined = pd.concat([interesting, total_water_sec_countries], axis=1, sort=False)
joined['water_sec'] = joined['water_sec_count'] / joined['total_count']
joined.index.name = 'ISO3'

In [20]:
# Join water data and sufficiencies
sufficiencies = sufficiencies.merge(joined['water_sec'], on='ISO3')

In [21]:
len(sufficiencies)

166

## Food Security
* Dataset: GFSI -- Global Food Security Index 2018, for 113 countries
* Source: EIU Global Food Security Index 2018 Building resilience in the face of rising food-security risks, 2018



In [22]:
path_to_file = os.path.join(path_to_datasets,'Food_Security/GFSI_2018_min.csv')
food_secu = pd.read_excel(path_to_file)

In [23]:
food_secu.head()

,Country_name,GFSI_2018
0,Singapore,85.9
1,Ireland,85.5
2,United Kingdom,85.0
3,United States,85.0
4,Netherlands,84.7


In [24]:
food_secu['GFSI_2018'] = food_secu['GFSI_2018']/100

In [25]:
len(food_secu)

113

In [26]:
## Join with Sufficiencies
sufficiencies = sufficiencies.merge(food_secu, left_on='name', right_on='Country_name', how='left').drop('Country_name', axis=1)

In [27]:
len(sufficiencies)

166

## Malnutrition
To download from https://data.worldbank.org/indicator/SH.STA.MALN.ZS


In [28]:
malnutrition_data = pd.read_csv(path_to_datasets + 'Malnutrition/API_SH.STA.MALN.ZS_DS2_en_csv_v2_49604.csv', header=2)
# get last valid value (transpose than ffill than retranspose)
malnutrition_filled = malnutrition_data.T.fillna(method='ffill').T[['Country Code', '2018']].set_index('Country Code')
malnutrition_filled.index.name = 'ISO3'
# keep only numerical values
malnutrition_df = malnutrition_filled[malnutrition_filled['2018'].apply(lambda x: type(x) == float)].rename({'2018': 'malnutrition'}, axis=1)/100

In [29]:
# Join 
sufficiencies = sufficiencies.join(malnutrition_df, how='left', on='ISO3')

In [30]:
sufficiencies

,ISO3,2000,ssp1,ssp2,ssp3,ssp4,ssp5,trade_dep_2013,name,water_sec,GFSI_2018,malnutrition
0,AFG,0.712556,0.536646,0.468967,0.390449,0.367319,0.655464,0.269311,Afghanistan,0.652642,NaN,0.25
1,AGO,0.591377,0.453087,0.571087,0.381656,0.558107,0.686487,0.221301,Angola,0.010298,0.386,0.19
2,ALB,2.329763,2.544620,2.428040,2.353029,2.508106,2.448250,0.208123,Albania,0.390023,NaN,0.063
3,ARE,0.417912,0.513526,0.515186,0.525990,0.529507,0.520904,0.000000,United Arab Emirates,0.998895,0.725,NaN
4,ARG,7.931488,5.890471,6.701443,6.249030,6.205168,6.367101,1.000000,Argentina,0.195395,0.692,0.023
5,ARM,1.878032,2.100658,2.219366,1.968406,2.177998,2.338979,0.124763,Armenia,0.527840,NaN,0.026
6,AUS,10.710803,4.663941,4.728667,8.695204,5.828738,4.555344,1.000000,Australia,0.246704,0.837,0.002
7,AUT,2.429237,2.164068,2.273159,2.369145,2.180175,2.220470,0.326643,Austria,NaN,0.821,NaN
8,AZE,2.136397,1.791686,1.848153,1.695168,1.818989,1.737322,0.221751,Azerbaijan,0.362187,0.582,0.049
9,BDI,0.458729,0.299079,0.299656,0.246161,0.248014,0.331084,NaN,Burundi,NaN,0.239,0.293


In [31]:
sufficiencies.to_csv(path_to_datasets + 'sufficiencies_added_data.csv', index=False)

# Scatterplots

## Load/prepare sufficiencies df

In [25]:
# Cases: all_ssp column
#0: down in all scenario (Red)
#1: down in most scenario (Orange)

# in 2/3 scenarios
#2: variable (Pink)

# in 4 scenarios
#3: up in most scenario (Blue-Green)
#4: up in all scenario (Green)

consistency = pd.read_csv(path_to_datasets + 'suff_map.csv')[['ISO3', 'all_ssp']]
consistency.head()

,ISO3,all_ssp
0,AFG,1
1,AGO,2
2,ALB,2
3,ARE,2
4,ARG,0


In [101]:
sufficiencies = pd.read_csv(path_to_datasets + 'sufficiencies_added_data.csv')
sufficiencies = sufficiencies.dropna(subset=['ssp1','ssp2','ssp3','ssp4','ssp5'])
sufficiencies = sufficiencies.rename(index=str, columns={"trade_dep_2013": "trade",
                               "GFSI_2018":"GFSI",
                               "2000":"CalSuff_2000" })

# Singapore
sufficiencies['trade'][132] = 0.05

# water_sec = % water depletion
# 'water' = % water security
sufficiencies['water'] = 1 - sufficiencies['water_sec']
sufficiencies = sufficiencies.drop('water_sec', axis=1)


# Cases: all_ssp column
#0: down in all scenario 
#1: down in most scenario 
#2: variable 
#3: up in most scenario 
#4: up in all scenario
sufficiencies = sufficiencies.merge(consistency, on='ISO3').rename({'all_ssp': 'delta_consistency'}, axis=1)

sufficiencies['delta_CalSuff_avg'] = (sufficiencies['ssp1']
                        +sufficiencies['ssp2']
                        +sufficiencies['ssp3']
                        +sufficiencies['ssp4']
                        +sufficiencies['ssp5'])/5 - sufficiencies['CalSuff_2000']

sufficiencies['delta_max'] = np.nan
for i, row in sufficiencies.iterrows():
    # (signed change, absolute change)
    changes = list([(row['ssp1'] - row['CalSuff_2000'], abs(row['ssp1'] - row['CalSuff_2000'])),
                    (row['ssp2'] - row['CalSuff_2000'], abs(row['ssp2'] - row['CalSuff_2000'])),
                    (row['ssp3'] - row['CalSuff_2000'], abs(row['ssp3'] - row['CalSuff_2000'])),
                    (row['ssp4'] - row['CalSuff_2000'], abs(row['ssp4'] - row['CalSuff_2000'])),
                    (row['ssp5'] - row['CalSuff_2000'], abs(row['ssp5'] - row['CalSuff_2000']))])
    # take signed change where absolute change is max
    sufficiencies.loc[i, 'delta_max'] = max(changes, key=operator.itemgetter(1))[0]


    
sufficiencies['lowest_ssp'] = np.nan
for i, row in sufficiencies.iterrows():
    # (signed change, absolute change)
    
    changes = list([row['ssp1'],
                    row['ssp2'],
                    row['ssp3'],
                    row['ssp4'],
                    row['ssp5']])
    sufficiencies.loc[i, 'lowest_ssp'] = min(changes)




sufficiencies.head()

/Users/Romain/anaconda/envs/project/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,ISO3,CalSuff_2000,ssp1,ssp2,ssp3,ssp4,ssp5,trade,name,GFSI,malnutrition,water,delta_consistency,delta_CalSuff_avg,delta_max,lowest_ssp
0,AFG,0.712556,0.536646,0.468967,0.390449,0.367319,0.655464,0.269311,Afghanistan,NaN,0.250,0.347358,1,-0.228788,-0.345237,0.367319
1,AGO,0.591377,0.453087,0.571087,0.381656,0.558107,0.686487,0.221301,Angola,0.386,0.190,0.989702,2,-0.061292,-0.209721,0.381656
2,ALB,2.329763,2.544620,2.428040,2.353029,2.508106,2.448250,0.208123,Albania,NaN,0.063,0.609977,2,0.126646,0.214857,2.353029
3,ARE,0.417912,0.513526,0.515186,0.525990,0.529507,0.520904,0.000000,United Arab Emirates,0.725,NaN,0.001105,2,0.103111,0.111595,0.513526
4,ARG,7.931488,5.890471,6.701443,6.249030,6.205168,6.367101,1.000000,Argentina,0.692,0.023,0.804605,0,-1.648846,-2.041017,5.890471


In [102]:
sufficiencies[sufficiencies['name'] == 'Switzerland']

,ISO3,CalSuff_2000,ssp1,ssp2,ssp3,ssp4,ssp5,trade,name,GFSI,malnutrition,water,delta_consistency,delta_CalSuff_avg,delta_max,lowest_ssp
25,CHE,1.0373,1.147859,1.227865,1.420269,1.257557,1.171975,0.204969,Switzerland,0.835,NaN,NaN,4,0.207805,0.382969,1.147859


## Fct to make scatterplots

In [103]:
def plot_scatter(x,y):
    p = figure()

    palette=palettes.Category20[len(sufficiencies['Category'].unique())]
    color_map = bmo.CategoricalColorMapper(factors=sufficiencies['Category'].unique(),
                                       palette=palette)


    p.scatter(x=x, y=y,
              color={'field': 'Category', 'transform': color_map},
              legend='Category', source=source,size=7)


    #p.circle(x='2000', y='trade_dep_2013', size=10, source=source, color={'field': 'Category', 'transform': color_map})
    p.title.text = x+' vs '+y
    p.xaxis.axis_label = x
    p.yaxis.axis_label = y

    hover = HoverTool()
    hover.tooltips=[('Country', '@name'),
                    (x, '@'+x),
                    (y, '@'+y)]
    p.add_tools(hover)

    show(p)

## Iterate on thresholds

In [107]:
# Add Category column 

def custom_categorization(row):
    if row['CalSuff_2000'] > 3 and row['trade'] >0.8: 
        return 'Exporters' # e.g Brazil, USA, France, Denmark (Greenland)
    elif row['delta_CalSuff_avg'] > 0.6 and row['delta_consistency'] >= 3:
        return 'Winners'
    elif row['trade'] < 0.1:
        return 'Importers'
    elif (row['lowest_ssp'] < 1 or row['CalSuff_2000'] < 1.5) and row['delta_CalSuff_avg'] < -0.2 and row['delta_consistency'] <= 1:
          return 'In danger (drop below 1)'
    elif row['CalSuff_2000'] < 1.2 and row['delta_CalSuff_avg'] < 0.2 and row['delta_consistency'] <= 1:
          return 'Dark spots still bad'
    elif row['CalSuff_2000'] < 1.2 and row['delta_CalSuff_avg'] >= 0.2:
          return 'Dark spots getting better'
    elif row['delta_consistency'] == 2:
            return 'quite variable'
    elif row['delta_consistency'] <= 2 and row['delta_CalSuff_avg'] < -0.1 and row['CalSuff_2000'] > 2:
        return 'decreasing'
    elif row['delta_consistency'] <= 2 and row['delta_CalSuff_avg'] < -0.1:
        return 'kinda in danger'

    
    #elif row['malnutrition'] > 0.1 :
    #      return 'Malnurished'
    return 'Outliers'

sufficiencies['Category'] = sufficiencies.apply (lambda row: custom_categorization(row), axis=1)


# Re-load sufficiencies for Bokeh

source = ColumnDataSource(sufficiencies)


In [108]:
df = pd.DataFrame(sufficiencies.groupby('Category')['name'].apply(list))
for i, row in df.iterrows():
    print(i, list(row))
    print()

Dark spots getting better [['Central African Republic', 'Switzerland', 'Morocco', 'Peru', 'North Korea']]

Dark spots still bad [['Burundi', 'Bangladesh', 'Republic of Congo', 'Dominican Republic', 'Egypt', 'Niger', 'Rwanda']]

Exporters [['Argentina', 'Australia', 'Bulgaria', 'Belize', 'Brazil', 'Canada', 'Denmark', 'France', 'Guyana', 'Hungary', 'Kazakhstan', 'Lithuania', 'Latvia', 'Paraguay', 'Ukraine', 'Uruguay', 'United States of America']]

Importers [['United Arab Emirates', 'Belgium', 'Brunei', 'Northern Cyprus', 'Cyprus', 'Djibouti', 'Algeria', 'Gambia', 'Iraq', 'Israel', 'Jordan', 'Japan', 'South Korea', 'Kuwait', 'Lebanon', 'Montenegro', 'Mauritania', 'Netherlands', 'Saudi Arabia', 'Singapore', 'Trinidad and Tobago', 'Taiwan', 'Yemen']]

In danger (drop below 1) [['Afghanistan', 'Benin', 'Burkina Faso', 'Bhutan', 'Costa Rica', 'Eritrea', 'Ethiopia', 'Ghana', 'Guinea Bissau', 'Equatorial Guinea', 'Guatemala', 'Honduras', 'Indonesia', 'Kenya', 'Sri Lanka', 'Madagascar', 'Mexic

In [109]:
for x in ['CalSuff_2000','delta_CalSuff_avg', 'delta_max', 'delta_consistency']:
    for y in ['water','GFSI','trade','malnutrition']:
          plot_scatter(x,y)